In [1]:
%load_ext autoreload
%autoreload 2

import os 
import sys
import pandas as pd
from transformers import AutoTokenizer
from telethon import TelegramClient

sys.path.append(os.path.abspath('../src'))  # to import modules from the src/service directory
from services.telegram_scrapper import run_scrapper
from utils.utils import clean_data, normalize_data, hf_tokenize, regex_tokenize

sys.path.append(os.path.abspath('../config'))
from config import load_credentials




# Reaload Data From Telegram but with views data

In [2]:
creds = load_credentials() # load credentials from environment

# initializing client
client = TelegramClient("scraping_session;.session", creds["api_id"], creds["api_hash"])

#await run_scrapper(client) #run the entry point function of the telegram_scrapper 
print("Scrapped Data successfully ✅")

Scrapped Data successfully ✅


In [3]:
data = pd.read_csv("../data/raw/telegram_WithViews_data.csv") # read the scrapped data as a dataframe

In [4]:
data.rename(columns={'Message': 'Views', 'Views': 'Message'}, inplace=True)
data


,Channel Title,Channel Username,ID,Views,Message,Date,Media Path
0,Sheger online-store,@Shageronlinestore,7437,1773.0,Waterproof Baby Urine Mat Cover\n\nውሃ የማያስገባ ...,2025-06-25 14:47:35+00:00,../data/raw/photo\@Shageronlinestore_7437.jpg
1,Sheger online-store,@Shageronlinestore,7436,1750.0,NaN,2025-06-25 14:47:35+00:00,../data/raw/photo\@Shageronlinestore_7436.jpg
2,Sheger online-store,@Shageronlinestore,7435,1750.0,NaN,2025-06-25 14:47:35+00:00,../data/raw/photo\@Shageronlinestore_7435.jpg
3,Sheger online-store,@Shageronlinestore,7434,1742.0,NaN,2025-06-25 14:47:35+00:00,../data/raw/photo\@Shageronlinestore_7434.jpg
4,Sheger online-store,@Shageronlinestore,7433,2620.0,2 tier stainless still dish drainer rack\n\n...,2025-06-25 09:56:08+00:00,NaN
...,...,...,...,...,...,...,...
2995,NEVA COMPUTER®,@nevacomputer,8100,2282.0,NaN,2023-11-28 05:55:47+00:00,../data/raw/photo\@nevacomputer_8100.jpg
2996,NEVA COMPUTER®,@nevacomputer,8099,1888.0,𝗡𝗘𝗪 𝗔𝗥𝗥𝗜𝗩𝗔𝗟 from \n\n\n🅑🅡🅐🅝🅓 : Dell inspiron \...,2023-11-28 05:55:47+00:00,../data/raw/photo\@nevacomputer_8099.jpg
2997,NEVA COMPUTER®,@nevacomputer,8098,NaN,NaN,2023-11-28 05:51:37+00:00,NaN
2998,NEVA COMPUTER®,@nevacomputer,8097,2178.0,NaN,2023-11-28 05:50:59+00:00,../data/raw/photo\@nevacomputer_8097.jpg


In [5]:
print(f"Total number of messages scrapped: {data.shape[0]}") # print the total number of messages scrapped
print(f"with columns:")
for col in data.columns:
    print(col + "\n")
print(f"We can see that we have {len(data.columns)} columns✅")

Total number of messages scrapped: 3000
with columns:
Channel Title

Channel Username

ID

Views

Message

Date

Media Path

We can see that we have 7 columns✅


In [6]:
data.isna().sum() # check for missing values in the dataframe 

Channel Title          0
Channel Username       0
ID                     0
Views                  3
Message             1222
Date                   0
Media Path           135
dtype: int64

In [7]:
msg_with_no_photo = data[(data["Message"].isna() == False) & (data["Media Path"].isna())].shape[0] # check for messages that have a message but no media path(picture)
print(f"There are {msg_with_no_photo} messages in the dataset without photo")

There are 123 messages in the dataset without photo


# The next task is to clean the Dataset

In [8]:
cleaned_data = clean_data(data) # clean the data by removing duplicates and messages with no text and media


In [9]:
print(f"Total number of messages after cleaning: {cleaned_data.shape[0]}") # print the total number of messages after cleaning
cleaned_data.isna().sum() # check for missing values in the cleaned dataframe

Total number of messages after cleaning: 1778


Channel Title       0
Channel Username    0
ID                  0
Views               0
Message             0
Date                0
Media Path          0
dtype: int64

# Data Normalization

In [10]:
cleaned_normalized_data = normalize_data(cleaned_data)
cleaned_normalized_data.reset_index(drop=True, inplace=True)
cleaned_normalized_data


,Channel Title,Channel Username,ID,Views,Message,Date,Media Path,hashtags
0,Sheger online-store,@Shageronlinestore,7437,1773.0,Waterproof Baby Urine Mat Cover ውሃ የማያስገባ እንደገ...,2025-06-25 14:47:35+00:00,../data/raw/photo\@Shageronlinestore_7437.jpg,[ዛም_ሞል]
1,Sheger online-store,@Shageronlinestore,7433,2620.0,2 tier stainless still dish drainer rack የታጠቡ ...,2025-06-25 09:56:08+00:00,Not Available,[ዛም_ሞል]
2,Sheger online-store,@Shageronlinestore,7431,2265.0,2 tier stainless still dish drainer rack የታጠቡ ...,2025-06-25 09:56:08+00:00,../data/raw/photo\@Shageronlinestore_7431.jpg,[ዛም_ሞል]
3,Sheger online-store,@Shageronlinestore,7426,2591.0,የልብስ ማስጫ (ማድረቂያ) Three Layers of clothes hange...,2025-06-25 06:42:53+00:00,../data/raw/photo\@Shageronlinestore_7426.jpg,[ዛም_ሞል]
4,Sheger online-store,@Shageronlinestore,7423,3134.0,5.5L Glass dispenser jar with Bamboo stand ለተለ...,2025-06-24 15:47:54+00:00,../data/raw/photo\@Shageronlinestore_7423.jpg,[ዛም_ሞል]
...,...,...,...,...,...,...,...,...
1773,NEVA COMPUTER®,@nevacomputer,8106,2187.0,The 16GB DDR4 RAM Running @ 3200MHz and the Ei...,2023-11-28 12:29:04+00:00,../data/raw/photo\@nevacomputer_8106.jpg,[no tag]
1774,NEVA COMPUTER®,@nevacomputer,8105,2011.0,This Alienware m15 R5 Ryzen Edition Gaming Lap...,2023-11-28 12:27:58+00:00,../data/raw/photo\@nevacomputer_8105.jpg,[no tag]
1775,NEVA COMPUTER®,@nevacomputer,8103,2010.0,The Alienware m15 Ryzen Edition R5 is engineer...,2023-11-28 12:15:47+00:00,../data/raw/photo\@nevacomputer_8103.jpg,[no tag]
1776,NEVA COMPUTER®,@nevacomputer,8102,1960.0,Dell Alienware M15 R5 15.6'' QHD Gaming Laptop...,2023-11-28 12:13:18+00:00,../data/raw/photo\@nevacomputer_8102.jpg,[no tag]


In [11]:
print(cleaned_normalized_data.columns)
print(f"we have added a new '{cleaned_normalized_data.columns[-1]}' column to the dataframe✅")

Index(['Channel Title', 'Channel Username', 'ID', 'Views', 'Message', 'Date',
       'Media Path', 'hashtags'],
      dtype='object')
we have added a new 'hashtags' column to the dataframe✅


# Next step is to create a Rich Profile for each Vendor(channal) in our dataset

## Vendor Channel Activity & Consistency Analysis

This analysis computes the **posting frequency** for each vendor channel in your Telegram dataset, providing insight into their business activity and consistency.

- **Posting Frequency (Average Posts per Week):**  
  For each vendor channel, we calculate the total number of posts and the time span (in weeks) between their first and last post. The average posts per week is then computed as:
  

In [12]:
# Calculate Posting Frequency (Average Posts per Week) for Each Vendor Channel

import pandas as pd

# Assume your cleaned and normalized data is in 'cleaned_normalized_data'
# and has at least these columns: 'Channel', 'Date' (or 'Timestamp')

# Convert the date column to datetime if not already
cleaned_normalized_data['Date'] = pd.to_datetime(cleaned_normalized_data['Date'])

# Group by channel and calculate posting frequency
vendor_stats = (
    cleaned_normalized_data
    .groupby('Channel Title')
    .agg(
        total_posts=('Message', 'count'),
        first_post=('Date', 'min'),
        last_post=('Date', 'max')
    )
)

# Calculate number of weeks active for each channel
vendor_stats['weeks_active'] = ((vendor_stats['last_post'] - vendor_stats['first_post']).dt.days / 7).clip(lower=1)

# Calculate average posts per week
vendor_stats['avg_posts_per_week'] = vendor_stats['total_posts'] / vendor_stats['weeks_active']

# Display the results
vendor_stats = vendor_stats.sort_values('avg_posts_per_week', ascending=False)
vendor_stats[['total_posts', 'weeks_active', 'avg_posts_per_week']]

,total_posts,weeks_active,avg_posts_per_week
Channel Title,,,
Sheger online-store,274,11.571429,23.679012
HellooMarket,576,50.571429,11.389831
Shewa Brand,512,91.571429,5.591264
NEVA COMPUTER®,180,81.571429,2.206655
Fashion tera,236,120.428571,1.959668


In [13]:
# Create a nested dictionary to hold stats for each channel, rounding floats to 2 decimal places
import numpy as np
cols = ['total_posts', 'weeks_active', 'avg_posts_per_week']

channel_stats_dict = {
    channel: {
        col: float(f"{row[col]:.2f}") if isinstance(row[col], (float, np.floating)) else int(row[col])
        for col in cols
    }
    for channel, row in vendor_stats[cols].iterrows()
}

# print the stats for one channel
for channel, stats in list(channel_stats_dict.items())[:3]:
    print(f"{channel}: {stats}")

Sheger online-store: {'total_posts': 274.0, 'weeks_active': 11.57, 'avg_posts_per_week': 23.68}
HellooMarket: {'total_posts': 576.0, 'weeks_active': 50.57, 'avg_posts_per_week': 11.39}
Shewa Brand: {'total_posts': 512.0, 'weeks_active': 91.57, 'avg_posts_per_week': 5.59}


## Market Reach & Engagement Analysis

This section evaluates each vendor channel’s ability to reach and engage potential customers on Telegram.

- **Average Views per Post:**  
  For each channel, we compute the mean number of views across all their posts. This metric reflects the typical audience size for a vendor’s content.

- **Top Performing Post:**  
  For each vendor, we identify the post with the highest view count. This helps highlight their most successful product or promotional message, and can provide insight into what content resonates most with their audience.

**Interpretation:**
- Channels with high average views per post have greater market reach and visibility.
- The top performing post can be analyzed for content, timing, or format to inform future marketing strategies.

In [14]:
# Market Reach & Engagement Analysis

# 1. Calculate average views per post for each channel
avg_views = (
    cleaned_normalized_data
    .groupby('Channel Title')['Views']
    .mean()
    .round(2)
    .to_dict()
)

# 2. Identify the top performing post (highest view count) for each channel
top_posts = (
    cleaned_normalized_data
    .loc[cleaned_normalized_data.groupby('Channel Title')['Views'].idxmax()]
    .set_index('Channel Title')
)

# Add these metrics channel_stats_dict
for channel in channel_stats_dict:
    # Average views per post
    channel_stats_dict[channel]['avg_views_per_post'] = float(avg_views.get(channel, 0))
    # Top performing post info
    if channel in top_posts.index:
        channel_stats_dict[channel]['top_post'] = {
            'views': int(top_posts.loc[channel, 'Views']),
            'message': top_posts.loc[channel, 'Message'],
            'date': str(top_posts.loc[channel, 'Date'])
        }
    else:
        channel_stats_dict[channel]['top_post'] = None

# Example: print stats for a few channels
for channel, stats in list(channel_stats_dict.items())[:3]:
    print(f"{channel}:")
    print(f"  Avg Views/Post: {stats['avg_views_per_post']}")
    print(f"  Top Post: {stats['top_post']}")

Sheger online-store:
  Avg Views/Post: 6285.18
  Top Post: {'views': 9200, 'message': 'momcoc non stick cookware የማይዙ ማራኪ ግራናይት ቅብ ድስቶች 4 ድስቶች 1 መጥበሻ ከነ ክዳናቸው እንዲሁም ሁለት ጭልፋዎች በአንድ ላይ ዋጋ፦ 8,800 ብር ውስን ፍሬ ነው ያለን/ Limited Stock አድራሻ ቁ.2 መገናኛ ስሪ ኤም ሲቲ ሞል ሁለተኛ ፎቅ ቢሮ ቁ. SL-05A(ከ ሊፍቱ ፊት ለ ፊት) ቁ.2 ለቡ መዳህኒዓለም ቤተ/ክርስቲያን ፊት ለፊት 2ኛ ፎቅ ቢሮ ቁጥር.214 ለቡ ቅርንጫፍ0973611819 0909522840 0923350054 በTelegram ለማዘዝ ይጠቀሙ @shager_onlinestore ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን https://t.me/Shageronlinestore', 'date': '2025-04-13 07:09:42+00:00'}
HellooMarket:
  Avg Views/Post: 3781.72
  Top Post: {'views': 8162, 'message': 'ድካምዎን በብርታት ቀይረው ከወዳጆቾ ጋር እየተጫዎቱ የሚጠጡት ቡና ለማዘዝ 0974312223 ይደውሉ ወይም https://t.me/helloo_market_bot?start=172610001 ይጠቀሙ! መጠን: 1kg', 'date': '2024-10-27 18:02:21+00:00'}
Shewa Brand:
  Avg Views/Post: 14715.82
  Top Post: {'views': 49336, 'message': 'adidas SUPERSTAR Size 36 MADE IN INDONESIA SHEWA BRAND የቴሌግራም ቻናላችንን ይቀላቀሉ https://t.me//shewabrand https://t.me//shewabrand https://t.me//shewabrand https://t.

## Product and Price Extraction from Top Posts

In this step, we leverage our fine-tuned NER model to extract key business information—specifically, the **product name** and its **price**—from each vendor channel's top-performing post.

- **Product Extraction:**  
  For each channel's most-viewed post, we use the NER pipeline to identify and extract the first entity tagged as a product.

- **Price Extraction:**  
  The NER pipeline also identifies price entities, which may appear in various formats (e.g., "ዋጋ:-550ብር", "price -600", etc.).  
  We use a regular expression to robustly extract the numeric value from the price string, converting it to an integer for consistency and further analysis.

**Interpretation:**  
- This process allows us to automatically build a structured summary of each vendor's most popular product and its listed price, enabling downstream analytics such as price comparisons, product popularity tracking, and market trend analysis.

In [15]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_path = "../models/my_xml_ner_model"  # e.g., "./saved_model/ner_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

ner_pipeline = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

Device set to use cpu


In [27]:
import re

def extract_product_and_price(text, ner_pipeline):
    """
    Extracts the product and price from a given text using the NER pipeline.
    Returns a tuple: (product, price_int or None)
    """
    entities = ner_pipeline(text)
    product = None
    price = None

    # Find product entity (first occurrence)
    for ent in entities:
        if ent['entity_group'].lower() == 'product':
            product = ent['word']
            break

    # Find price entity (first occurrence)
    for ent in entities:
        if ent['entity_group'].lower() == 'price':
            # Extract number from the price string using regex
            match = re.search(r'[-+]?\d[\d,]*', ent['word'])
            if match:
                # Remove commas and convert to int
                price = int(match.group(0).replace(',', ''))
            break

    return product, price

# Example usage for all channels' top posts:
for channel, stats in channel_stats_dict.items():
    top_post = stats.get('top_post')
    if top_post and top_post['message']:
        product, price = extract_product_and_price(top_post['message'], ner_pipeline)
        channel_stats_dict[channel]['top_post']['product'] = product
        channel_stats_dict[channel]['top_post']['price'] = price

# Print example
for channel, stats in list(channel_stats_dict.items())[:3]:
    print(f"{channel}:")
    print(f"  Product: {stats['top_post'].get('product')}")
    print(f"  Price: {stats['top_post'].get('price')}")

Sheger online-store:
  Product: momcoc non stick cookware
  Price: 8800
HellooMarket:
  Product: ቡና
  Price: None
Shewa Brand:
  Product: adidas SUPERSTAR
  Price: None


In [28]:
# Print example
for channel, stats in list(channel_stats_dict.items())[:]:
    print(f"{channel}:")
    print(f"  Product: {stats['top_post'].get('product')}")
    print(f"  Price: {stats['top_post'].get('price')}")

Sheger online-store:
  Product: momcoc non stick cookware
  Price: 8800
HellooMarket:
  Product: ቡና
  Price: None
Shewa Brand:
  Product: adidas SUPERSTAR
  Price: None
NEVA COMPUTER®:
  Product: REPUBLIC OF GAMERS STANDARD
  Price: None
Fashion tera:
  Product: VIGUER Automatic Watches (Men)
  Price: 3500


None highlights products that price was not specified for

## Business Profile: Average Price Point

This step uses your fine-tuned NER model to extract and analyze the **average price point** for each vendor channel.

- **How it works:**  
  For every message in each channel, the NER model is used to extract price entities. All detected prices are converted to integers and averaged for each channel.

- **Interpretation:**  
  - **High average price point:** The vendor may be a low-volume, high-margin seller (e.g., luxury goods).
  - **Low average price point:** The vendor may be a high-volume, low-margin seller (e.g., everyday items).

This metric helps you profile vendors by their typical product pricing and market positioning.

In [24]:
import re

def extract_price(text, ner_pipeline):
    """
    Extracts the first price entity from text using the NER pipeline.
    Returns the price as an integer if found, else None.
    """
    entities = ner_pipeline(text)
    for ent in entities:
        if ent['entity_group'].lower() == 'price':
            match = re.search(r'[-+]?\d[\d,]*', ent['word'])
            if match:
                return int(match.group(0).replace(',', ''))
    return None

# For each channel, calculate the average price and add it to channel_stats_dict
for channel, group in cleaned_normalized_data.groupby('Channel Title'):
    prices = []
    for msg in group['Message']:
        price = extract_price(str(msg), ner_pipeline)
        if price is not None:
            if channel == "Fashion tera":
                if price <20000:
                    prices.append(price)
                else:
                    continue
            elif channel != "Fashion tera":
                prices.append(price)
    avg_price = round(sum(prices) / len(prices), 2) if prices else None
    if channel in channel_stats_dict:
        channel_stats_dict[channel]['avg_price_point'] = avg_price

# Example: print stats for a few channels
for channel, stats in list(channel_stats_dict.items())[:3]:
    print(f"{channel}:")
    print(f"  Average Price Point: {stats.get('avg_price_point')}")

Sheger online-store:
  Average Price Point: 761.9
HellooMarket:
  Average Price Point: None
Shewa Brand:
  Average Price Point: None


In [25]:
# Example: print stats for a few channels
for channel, stats in list(channel_stats_dict.items())[:]:
    print(f"{channel}:")
    print(f"  Average Price Point: {stats.get('avg_price_point')}")

Sheger online-store:
  Average Price Point: 761.9
HellooMarket:
  Average Price Point: None
Shewa Brand:
  Average Price Point: None
NEVA COMPUTER®:
  Average Price Point: 59112.15
Fashion tera:
  Average Price Point: 3128.26


In [29]:
import numpy as np

# Gather all values for normalization
posts = [stats['avg_posts_per_week'] for stats in channel_stats_dict.values()]
views = [stats['avg_views_per_post'] for stats in channel_stats_dict.values()]
prices = [stats['avg_price_point'] for stats in channel_stats_dict.values() if stats['avg_price_point'] is not None]

def min_max(val, min_val, max_val):
    if max_val == min_val:
        return 0.0
    return (val - min_val) / (max_val - min_val)

min_posts, max_posts = min(posts), max(posts)
min_views, max_views = min(views), max(views)
min_price, max_price = (min(prices), max(prices)) if prices else (0, 1)

for channel, stats in channel_stats_dict.items():
    norm_posts = min_max(stats['avg_posts_per_week'], min_posts, max_posts)
    norm_views = min_max(stats['avg_views_per_post'], min_views, max_views)
    price = stats['avg_price_point']
    norm_price = min_max(price, min_price, max_price) if price is not None else None

    # If price is missing, only use posts and views (weights sum to 1)
    if norm_price is not None:
        lending_score = 0.3 * norm_posts + 0.3 * norm_views + 0.4 * norm_price
    else:
        lending_score = (0.5 * norm_posts + 0.5 * norm_views)  # weights sum to 1

    stats['lending_score'] = round(lending_score, 3)

# Example: print lending scores
for channel, stats in channel_stats_dict.items():
    print(f"{channel}: Lending Score = {stats['lending_score']}")

Sheger online-store: Lending Score = 0.369
HellooMarket: Lending Score = 0.217
Shewa Brand: Lending Score = 0.584
NEVA COMPUTER®: Lending Score = 0.406
Fashion tera: Lending Score = 0.083


## Lending Score: Combining Business Metrics

The **Lending Score** is a composite metric designed to help assess the lending potential of each vendor channel. It combines:

- **Activity:** Average posts per week (30%)
- **Engagement:** Average views per post (30%)
- **Average Price Point:** Typical product price (40%)

All metrics are normalized (min-max scaling) before combining.  
If a channel does not post prices, the score is based only on activity and engagement (each weighted at 50%).

**Interpretation:**
- A higher Lending Score suggests a more active, engaged, and higher-value vendor—potentially a better lending candidate.
- Channels with missing price data are still scored, but only on activity and engagement.

In [30]:
print("| Channel Name | Avg. Views/Post | Posts/Week | Avg. Price (ETB) | Lending Score |")
print("|--------------|-----------------|------------|------------------|--------------|")
for channel, stats in channel_stats_dict.items():
    avg_views = stats.get('avg_views_per_post', 'None')
    posts_week = stats.get('avg_posts_per_week', 'None')
    avg_price = stats.get('avg_price_point', 'None')
    lending_score = stats.get('lending_score', 'None')
    print(f"| {channel} | {avg_views} | {posts_week} | {avg_price} | {lending_score} |")

| Channel Name | Avg. Views/Post | Posts/Week | Avg. Price (ETB) | Lending Score |
|--------------|-----------------|------------|------------------|--------------|
| Sheger online-store | 6285.18 | 23.68 | 761.9 | 0.369 |
| HellooMarket | 3781.72 | 11.39 | None | 0.217 |
| Shewa Brand | 14715.82 | 5.59 | None | 0.584 |
| NEVA COMPUTER® | 3863.84 | 2.21 | 59112.15 | 0.406 |
| Fashion tera | 6215.8 | 1.96 | 3128.26 | 0.083 |


# Vendor score Board Table

| Channel Name           | Avg. Views/Post | Posts/Week | Avg. Price (ETB) | Lending Score |
|------------------------|-----------------|------------|------------------|--------------|
| Sheger online-store         |  6285.18          |   23.68     |     761.9       |    0.369     |
| HellooMarket           |  3781.72        |   11.39     |     None       |    0.217     |
| Shewa Brand           |  14715.82        |   5.59     |      None        |    0.584     |
| NEVA COMPUTER®         |  3863.84          |   2.21     |      59112.15        |    0.406     |
| Fashion tera         |  6215.8         |   1.96     |     3128.26       |    0.083     |